In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-au10vwje
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-au10vwje
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=ea6dda8d1c81eb2418bdaed02cb6f8c1ff723329eafb7b15230043fb0e0c6f81
  Stored in directory: /tmp/pip-ephem-wheel-cache-re82f3wz/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [23]:
import re
import nltk
import warnings
import pandas as pd 
from tqdm import tqdm
from nltk.corpus import stopwords                     
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences  
from keras.layers import Input, Dense, Flatten, Dropout, LSTM, Embedding, Bidirectional, BatchNormalization, TimeDistributed

warnings.filterwarnings("ignore", category = UserWarning, module = 'bs4')

In [24]:
df = pd.read_csv("data.csv")
df.columns = ['Phrase', 'Sentiment']

In [25]:
def clean(df):
  nltk.download('stopwords')
  for i in range(df.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', df['Phrase'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    df['Phrase'][i] = review
  return df

In [26]:
data = clean(df)
df = data
encoded_dict = {'negative': 0, 'neutral': 1, 'positive': 2}
df['Sentiment'] = df.Sentiment.map(encoded_dict)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
X = df.Phrase.values
y = df.Sentiment.values
y = to_categorical(y)
num_sentiments = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

In [48]:
unique_words = set()
len_max = 0

for sent in tqdm(X_train):
  unique_words.update(sent)
  if(len_max < len(sent)):
    len_max = len(sent)

print("\n")
print(len(list(unique_words)))
print(len_max)

100%|██████████| 4673/4673 [00:00<00:00, 463455.17it/s]



27
225


In [49]:
tokenizer = Tokenizer(num_words = len(list(unique_words)), oov_token = 'OOV', char_level = False)
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen = len_max, padding = 'post', truncating = 'post')
X_test = pad_sequences(X_test, maxlen = len_max, padding = 'post', truncating = 'post')

X_train.shape, X_test.shape

((4673, 225), (1169, 225))

In [50]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 2)

In [59]:
def bilstm():
  model = Sequential([
    Embedding(len(list(unique_words)), 300, input_length = len_max),
    Bidirectional(LSTM(128, return_sequences = True)),
    Bidirectional(LSTM(64, return_sequences = False)),
    Dense(128, activation = 'relu'),
    Dense(num_sentiments, activation = 'softmax')])
  
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [60]:
model1 = bilstm()
history = model1.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 15, batch_size = 256, callbacks = [es])

Epoch 1/15
19/19 [==============================] - 9s 199ms/step - loss: 0.9951 - accuracy: 0.5251 - val_loss: 0.9740 - val_accuracy: 0.5355
Epoch 2/15
19/19 [==============================] - 2s 125ms/step - loss: 0.9735 - accuracy: 0.5358 - val_loss: 0.9647 - val_accuracy: 0.5355
Epoch 3/15
19/19 [==============================] - 2s 125ms/step - loss: 0.9605 - accuracy: 0.5363 - val_loss: 0.9408 - val_accuracy: 0.5620
Epoch 4/15
19/19 [==============================] - 2s 127ms/step - loss: 0.9440 - accuracy: 0.5624 - val_loss: 0.9347 - val_accuracy: 0.5654
Epoch 5/15
19/19 [==============================] - 2s 124ms/step - loss: 0.9282 - accuracy: 0.5673 - val_loss: 0.9248 - val_accuracy: 0.5757
Epoch 6/15
19/19 [==============================] - 2s 126ms/step - loss: 0.9183 - accuracy: 0.5664 - val_loss: 0.9230 - val_accuracy: 0.5749
Epoch 7/15
19/19 [==============================] - 2s 125ms/step - loss: 0.9157 - accuracy: 0.5688 - val_loss: 0.9244 - val_accuracy: 0.5749
Epoch 